In [ ]:
import cloudscraper
from urllib.parse import unquote

scraper = cloudscraper.create_scraper()

# 1️⃣ Vào trang để lấy cookie
scraper.get("https://www.topcv.vn/viec-lam")

cookies = scraper.cookies.get_dict()

xsrf_cookie = cookies.get("XSRF-TOKEN")
session_cookie = cookies.get("topcv_session")

print("✅ Cookies:")
print("XSRF-TOKEN:", bool(xsrf_cookie))
print("topcv_session:", bool(session_cookie))
print("cf_clearance:", bool(cookies.get("cf_clearance")))

if not (xsrf_cookie and session_cookie):
    raise RuntimeError("❌ Thiếu cookie quan trọng")

# 2️⃣ Decode XSRF token cho header
xsrf_header = unquote(xsrf_cookie)

# 3️⃣ Headers CHUẨN
headers = {
    "accept": "application/json, text/plain, */*",
    "content-type": "application/json;charset=UTF-8",
    "origin": "https://www.topcv.vn",
    "referer": "https://www.topcv.vn/viec-lam",
    # "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/143",
    "x-xsrf-token": xsrf_header,
}

# 4️⃣ Payload
payload = {
    "page": 1,
    "limit": 20,
    "city": 0,
    "salary": None,
    "exp": None,
    "category": None,
    "reRanking": "[]"
}

# 5️⃣ Call API
resp_api = scraper.post(
    "https://www.topcv.vn/api-featured-jobs",
    json=payload,
    headers=headers,
)

# 6️⃣ Debug nếu lỗi
print("Status:", resp_api.status_code)
print("Response:", resp_api.text[:300])

resp_api.raise_for_status()

print("🎉 SUCCESS:", resp_api.json().keys())